# Setup

In [1]:
import os
import copy as cp
import numpy as np
import pandas as pd
from itertools import product
from circadian.models import Skeldon23
from circadian.lights import LightSchedule
from concurrent.futures import ThreadPoolExecutor

In [2]:
save_path = "../data/simulations/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [3]:
typical_indoor_lux_options = [0.03, 90, 180, 500, 1260, 9500]
ebook_order_options = ["ebook_first", "ebook_second"]
light_data_location = '../data/light_schedules'
forced_wakeup_light_threshold = 0.5 # lux
dt = 0.005 # hours

time = np.load(f"{light_data_location}/time.npy")
light_schedules = [np.load(f"{light_data_location}/regular_light.npy")]
light_schedule_names = ["regular_light"]

for lux_value in typical_indoor_lux_options:
    for ebook_order in ebook_order_options:
        name = f"chang14_light_indoor_lux_{lux_value}_{ebook_order}"
        light_schedules.append(
            np.load(f"{light_data_location}/{name}_light.npy")

        )
        light_schedule_names.append(name)

In [4]:
equilibration_days = 3
equilibration_loops = 10
lights_on = 6.0
lights_off = 22.0
typical_indoor_lux = 1000.0
equilibration_time = np.arange(0.0, 24 * equilibration_days, dt)
regular_schedule = LightSchedule.Regular(typical_indoor_lux, lights_on=lights_on,
                                         lights_off=lights_off)
equilibration_light = regular_schedule(equilibration_time)

# Simulation with reference parameters on regular schedule

In [5]:
reference_parameters_save_path = f"{save_path}/reference_parameters"
if not os.path.exists(reference_parameters_save_path):
    os.makedirs(reference_parameters_save_path)

In [6]:
reference_parameters = {'mu': 19.0, 'chi': 11.0, 'Delta': 6.0, 
                        'p': 0.6, 'tauc': 24.2, 'k': 0.55}

In [7]:
def simulate_light_schedules(idx, forced_wakeup_condition, parameters, initial_condition):
    print(f"Simulating idx: {idx}")
    light = light_schedules[idx]
    model = Skeldon23(params=parameters)
    trajectory = model.integrate(time, initial_condition=initial_condition, input=light)
    name = light_schedule_names[idx]
    np.savez(f"{reference_parameters_save_path}/{name}_{forced_wakeup_condition}_simulation.npz",
             parameters=parameters,
             initial_condition=initial_condition,
             trajectory_states=trajectory.states,
             sleep=model.sleep_state,
             dlmos=model.dlmos(),
             )

## With forced wake up by light

In [10]:
# Equilibration
parameters = cp.deepcopy(reference_parameters)
parameters['forced_wakeup_light_threshold'] = forced_wakeup_light_threshold
initial_condition = Skeldon23(params=parameters).equilibrate(equilibration_time, 
                                                             equilibration_light, 
                                                             num_loops=equilibration_loops,)

In [11]:
# Simulations
forced_wakeup_condition = "with_forced_wakeup"
pool_values = range(len(light_schedules))

with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda idx: simulate_light_schedules(
        idx, forced_wakeup_condition, parameters, initial_condition
    ), pool_values))

Simulating idx: 0
Simulating idx: 1
Simulating idx: 2
Simulating idx: 3
Simulating idx: 4
Simulating idx: 5
Simulating idx: 6
Simulating idx: 7
Simulating idx: 8
Simulating idx: 9
Simulating idx: 10


## Without forced wake up by light

In [12]:
# Equilibration
parameters = cp.deepcopy(reference_parameters)
parameters['forced_wakeup_light_threshold'] = None
initial_condition = Skeldon23(params=parameters).equilibrate(equilibration_time, 
                                                             equilibration_light, 
                                                             num_loops=equilibration_loops,)

In [13]:
# Simulations
forced_wakeup_condition = "without_forced_wakeup"
pool_values = range(len(light_schedules))

with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda idx: simulate_light_schedules(
        idx, forced_wakeup_condition, parameters, initial_condition
    ), pool_values))

Simulating idx: 0
Simulating idx: 1
Simulating idx: 2
Simulating idx: 3
Simulating idx: 4
Simulating idx: 5
Simulating idx: 6
Simulating idx: 7
Simulating idx: 8
Simulating idx: 9
Simulating idx: 10


# Parameter exploration

In [14]:
dt = 0.005 # hours
forced_wakeup_light_threshold = 0.5 # lux
# Default parameter values
default_mu = 19.0
default_Delta = 6.0
default_chi = 11.0
default_p = 0.6
default_k = 0.55
default_tauc = 24.2

In [10]:
sleep_exploration_save_path = f"{save_path}/sleep_parameter_exploration"
if not os.path.exists(sleep_exploration_save_path):
    os.makedirs(sleep_exploration_save_path)

circadian_exploration_save_path = f"{save_path}/circadian_parameter_exploration"
if not os.path.exists(circadian_exploration_save_path):
    os.makedirs(circadian_exploration_save_path)

In [16]:
def sleep_params_from_tuple(idx, param_tuples):
    return {'mu': param_tuples[idx][0], 'Delta': param_tuples[idx][1], 
            'chi': param_tuples[idx][2], 
            'p': default_p, 'k': default_k, 'tauc': default_tauc,
            'forced_wakeup_light_threshold': forced_wakeup_light_threshold}


def circadian_params_from_tuple(idx, param_tuples):
    return {'mu': default_mu, 'Delta': default_Delta, 'chi': default_chi, 
            'p': param_tuples[idx][0], 'k': param_tuples[idx][1], 
            'tauc': param_tuples[idx][2], 'forced_wakeup_light_threshold': forced_wakeup_light_threshold}


def simulate_parameter_exploration(idx, parameter_exploration_condition, save_path, param_tuples, params_from_tuple):
    print(f"Simulating idx: {idx}")
    parameters = params_from_tuple(idx, param_tuples)
    # equilibrate
    equilibration_model = Skeldon23(params=parameters)
    initial_condition = equilibration_model.equilibrate(equilibration_time, 
                                                        equilibration_light, 
                                                        num_loops=equilibration_loops)
    # simulate
    light = light_schedules[0] # regular
    model = Skeldon23(params=parameters)
    trajectory = model.integrate(time, initial_condition=initial_condition, input=light)
    name = light_schedule_names[0] # regular
    np.savez(f"{save_path}/{name}_{parameter_exploration_condition}_simulation_{idx}.npz",
             parameters=parameters,
             initial_condition=initial_condition,
             trajectory_states=trajectory.states,
             sleep=model.sleep_state,
             dlmos=model.dlmos(),
             )

## Sleep

In [17]:
mu_values = np.linspace(17.5, 19.5, 5)
delta_values = np.linspace(5.0, 9.0, 5)
chi_values = np.linspace(7.0, 11.0, 5)

param_tuples = list(product(mu_values, delta_values, chi_values))

# Simulations
parameter_exploration_condition = "sleep_parameter_exploration"
pool_values = range(len(param_tuples))

with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda idx: simulate_parameter_exploration(
        idx, parameter_exploration_condition, sleep_exploration_save_path, param_tuples, sleep_params_from_tuple 
    ), pool_values))

Simulating idx: 0Simulating idx: 1

Simulating idx: 2
Simulating idx: 3
Simulating idx: 4
Simulating idx: 5
Simulating idx: 6
Simulating idx: 7
Simulating idx: 8
Simulating idx: 9
Simulating idx: 10
Simulating idx: 11
Simulating idx: 12
Simulating idx: 13
Simulating idx: 14
Simulating idx: 15
Simulating idx: 16
Simulating idx: 17
Simulating idx: 18
Simulating idx: 19
Simulating idx: 20
Simulating idx: 21
Simulating idx: 22
Simulating idx: 23
Simulating idx: 24
Simulating idx: 25
Simulating idx: 26
Simulating idx: 27
Simulating idx: 28
Simulating idx: 29
Simulating idx: 30
Simulating idx: 31
Simulating idx: 32
Simulating idx: 33
Simulating idx: 34
Simulating idx: 35
Simulating idx: 36
Simulating idx: 37
Simulating idx: 38
Simulating idx: 39
Simulating idx: 40
Simulating idx: 41
Simulating idx: 42
Simulating idx: 43
Simulating idx: 44
Simulating idx: 45
Simulating idx: 46
Simulating idx: 47
Simulating idx: 48
Simulating idx: 49
Simulating idx: 50
Simulating idx: 51
Simulating idx: 52
Sim

## Circadian

In [18]:
p_values = np.linspace(0.5, 0.7, 5)
k_values = np.linspace(0.4, 0.7, 5)
tauc_values = np.linspace(23.8, 24.4, 5)

param_tuples = list(product(p_values, k_values, tauc_values))

# Simulations
parameter_exploration_condition = "circadian_parameter_exploration"
pool_values = range(len(param_tuples))

with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda idx: simulate_parameter_exploration(
        idx, parameter_exploration_condition, circadian_exploration_save_path, param_tuples, circadian_params_from_tuple 
    ), pool_values))

Simulating idx: 0
Simulating idx: 1
Simulating idx: 2
Simulating idx: 3
Simulating idx: 4
Simulating idx: 5
Simulating idx: 6
Simulating idx: 7
Simulating idx: 8
Simulating idx: 9
Simulating idx: 10
Simulating idx: 11
Simulating idx: 12
Simulating idx: 13
Simulating idx: 14
Simulating idx: 15
Simulating idx: 16
Simulating idx: 17
Simulating idx: 18
Simulating idx: 19
Simulating idx: 20
Simulating idx: 21
Simulating idx: 22
Simulating idx: 23
Simulating idx: 24
Simulating idx: 25
Simulating idx: 26
Simulating idx: 27
Simulating idx: 28
Simulating idx: 29
Simulating idx: 30
Simulating idx: 31
Simulating idx: 32
Simulating idx: 33
Simulating idx: 34
Simulating idx: 35
Simulating idx: 36
Simulating idx: 37
Simulating idx: 38
Simulating idx: 39
Simulating idx: 40
Simulating idx: 41
Simulating idx: 42
Simulating idx: 43
Simulating idx: 44
Simulating idx: 45
Simulating idx: 46
Simulating idx: 47
Simulating idx: 48
Simulating idx: 49
Simulating idx: 50
Simulating idx: 51
Simulating idx: 52
Sim

/Users/ftavella/arcascope/circadian/circadian/models.py:431: UserWarning: The model did not equilibrate. Try increasing the number of loops.
  warnings.warn("The model did not equilibrate. Try increasing the number of loops.")


Simulating idx: 84
Simulating idx: 85
Simulating idx: 86
Simulating idx: 87
Simulating idx: 88
Simulating idx: 89
Simulating idx: 90
Simulating idx: 91
Simulating idx: 92
Simulating idx: 93
Simulating idx: 94
Simulating idx: 95
Simulating idx: 96
Simulating idx: 97
Simulating idx: 98
Simulating idx: 99
Simulating idx: 100
Simulating idx: 101
Simulating idx: 102
Simulating idx: 103
Simulating idx: 104
Simulating idx: 105
Simulating idx: 106
Simulating idx: 107
Simulating idx: 108
Simulating idx: 109
Simulating idx: 110
Simulating idx: 111
Simulating idx: 112
Simulating idx: 113
Simulating idx: 114
Simulating idx: 115
Simulating idx: 116
Simulating idx: 117
Simulating idx: 118
Simulating idx: 119
Simulating idx: 120
Simulating idx: 121
Simulating idx: 122
Simulating idx: 123
Simulating idx: 124


# Filter out parameter exploration results

In [11]:
zone_of_interest = np.array([24.0 * 12.0 + 21.0, 24.0 * 13.0 + 9.0])
sleep_time_threshold = 6.5 # hours
number_of_switches_threshold = 2.0

In [12]:
exploration_condition_options = ["sleep_parameter_exploration", "circadian_parameter_exploration"]
data_path_options = [sleep_exploration_save_path, circadian_exploration_save_path]

for jdx in range(len(exploration_condition_options)):
    parameter_exploration_condition = exploration_condition_options[jdx]
    dir = data_path_options[jdx]

    files_in_dir = [f for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]
    files_in_dir = [f for f in files_in_dir if f.split(".")[1] == "npz"]
    results = []

    for filename in files_in_dir:
        idx = filename.split(".")[0].split("_")[-1]
        data = np.load(f"{dir}/{filename}", allow_pickle=True)
        parameters = data["parameters"].item()
        sleep = data["sleep"]
        states = data["trajectory_states"]
        dlmos = data["dlmos"]

        time_of_interest_mask = (time >= zone_of_interest[0]) & (time <= zone_of_interest[1])
        time_of_interest = time[time_of_interest_mask]
        sleep_of_interest = sleep[time_of_interest_mask]

        number_of_switches = np.sum(np.abs(np.diff(sleep_of_interest)))
        bedtime = time_of_interest[np.where(np.diff(sleep_of_interest) == 1)[0][0]] # % 24
        wake_up_time = time_of_interest[np.where(np.diff(sleep_of_interest) == -1)[0][-1]] # % 24
        sleep_time = wake_up_time - bedtime
        final_state = states[-1, :]
        dlmo = dlmos[-1] % 24

        if number_of_switches <= number_of_switches_threshold and sleep_time > sleep_time_threshold:
            results.append({
                'mu': parameters['mu'], 'Delta': parameters['Delta'], 'chi': parameters['chi'],
                'p': parameters['p'], 'k': parameters['k'], 'tauc': parameters['tauc'],
                'forced_wakeup_light_threshold': parameters['forced_wakeup_light_threshold'],
                'sleep_duration': sleep_time, 'sleep_onset': bedtime, 'sleep_offset': wake_up_time,
                'dlmo': dlmo, 'x_f': final_state[0], 'xc_f': final_state[1], 'n_f': final_state[2], 
                'H_f': final_state[3], 'S_f': sleep[-1], 'simulation_idx': idx,
            })

    selected_parameters = pd.DataFrame(results)
    selected_parameters = selected_parameters.set_index('simulation_idx')
    selected_parameters.to_csv(f"{dir}/{parameter_exploration_condition}_results.csv")
    print(f"Saving {len(selected_parameters)} parameters for {parameter_exploration_condition}")

Saving 61 parameters for sleep_parameter_exploration
Saving 125 parameters for circadian_parameter_exploration


# Simulate selected parameters on Chang14 schedule under two different indoor light conditions

In [13]:
time = np.load(f"{light_data_location}/time.npy")

save_dir_options = ["chang14_sleep_parameters", "chang14_circadian_parameters"]

for save_dir in save_dir_options:
    dir = f"{save_path}/{save_dir}"
    if not os.path.exists(dir):
        os.makedirs(dir)

In [15]:
def simulate_chang14(idx, time, light, selected_parameters, partial_save_path):
    print(f"Simulating idx: {idx}")
    mu = selected_parameters.loc[idx, 'mu']
    delta = selected_parameters.loc[idx, 'Delta']
    chi = selected_parameters.loc[idx, 'chi']
    p = selected_parameters.loc[idx, 'p']
    k = selected_parameters.loc[idx, 'k']
    tauc = selected_parameters.loc[idx, 'tauc']
    S0 = selected_parameters.loc[idx, 'S_f']
    forced_wakeup_light_threshold = selected_parameters.loc[idx, 'forced_wakeup_light_threshold']
    parameters = {
        'S0': S0, 'mu': mu, 'Delta': delta, 'chi': chi,
        'p': p, 'k': k, 'tauc': tauc,
        'forced_wakeup_light_threshold': forced_wakeup_light_threshold,
        }
    initial_condition = np.array([
        selected_parameters.loc[idx, 'x_f'],
        selected_parameters.loc[idx, 'xc_f'],
        selected_parameters.loc[idx, 'n_f'],
        selected_parameters.loc[idx, 'H_f'],
    ])
    model = Skeldon23(params=parameters)
    trajectory = model.integrate(time, initial_condition=initial_condition,
                                 input=light)

    np.savez(f"{partial_save_path}_{idx}.npz",
             parameters=parameters,
             trajectory_states=trajectory.states,
             sleep=model.sleep_state)

In [ ]:
baseline_lux_options = [90, 500]
exploration_condition_options = ["sleep_parameter_exploration", "circadian_parameter_exploration"]

for jdx in range(len(baseline_lux_options)):
    for kdx in range(len(exploration_condition_options)):
        for ebook_order in ebook_order_options:
            baseline_lux = baseline_lux_options[jdx]
            exploration_condition = exploration_condition_options[kdx]

            selected_parameters_dir = f"{save_path}/{exploration_condition}/{exploration_condition}_results.csv"
            selected_parameters = pd.read_csv(selected_parameters_dir, index_col=0)

            partial_save_path = f"{save_path}/{save_dir_options[kdx]}/"
            partial_save_path += f"baseline_lux_{baseline_lux_options[jdx]}_"
            partial_save_path += f"{ebook_order}_simulation"

            light = np.load(f"{light_data_location}/chang14_light_indoor_lux_{baseline_lux}_{ebook_order}_light.npy",
                            allow_pickle=True)

            pool_idx_values = selected_parameters.index.to_list()

            print(f"Simulating pool for {exploration_condition}, {baseline_lux} lux, {ebook_order}")
            with ThreadPoolExecutor() as executor:
                results = list(executor.map(lambda idx: simulate_chang14(
                    idx, time, light, selected_parameters, partial_save_path
                ), pool_idx_values))

Simulating pool for sleep_parameter_exploration, 90 lux, ebook_first
Simulating idx: 122
Simulating idx: 37
Simulating idx: 36
Simulating idx: 123
Simulating idx: 109
Simulating idx: 121
Simulating idx: 34
Simulating idx: 120
Simulating idx: 108
Simulating idx: 118
Simulating idx: 119Simulating idx: 32

Simulating idx: 33
Simulating idx: 54
Simulating idx: 83
Simulating idx: 96
Simulating idx: 82Simulating idx: 57

Simulating idx: 94
Simulating idx: 81
Simulating idx: 95
Simulating idx: 52
Simulating idx: 91
Simulating idx: 84
Simulating idx: 90
Simulating idx: 53
Simulating idx: 79
Simulating idx: 86
Simulating idx: 92
Simulating idx: 93
Simulating idx: 87
Simulating idx: 78
Simulating idx: 61
Simulating idx: 62
Simulating idx: 89
Simulating idx: 88
Simulating idx: 63
Simulating idx: 77
Simulating idx: 67
Simulating idx: 66
Simulating idx: 58
Simulating idx: 64
Simulating idx: 70
Simulating idx: 65
Simulating idx: 59
Simulating idx: 103
Simulating idx: 117
Simulating idx: 116
Simulati

# Simulate different initial conditions

In [ ]:
initial_conditions_save_path = '../data/different_initial_conditions'
if not os.path.exists(initial_conditions_save_path):
    os.makedirs(initial_conditions_save_path)

In [ ]:
days = 14.0
regular_indoor_lux = 1000.0
time = np.load(f"{light_data_location}/time.npy")

parameters = {
    'mu': 19.0,
    'Delta': 6.0,
    'chi': 11.0,
    'k': 0.55,
    'p': 0.6,
    'tauc': 24.2,
    'forced_wakeup_light_threshold': 0.5,
}

lights_on_start_times = [4, 5, 6, 7, 8, 9]
regular_light_options = []

for lights_on in lights_on_start_times:
    regular_light = LightSchedule.Regular(regular_indoor_lux, lights_on=lights_on, lights_off=int(np.mod(lights_on + 16, 24)))
    regular_light_options.append(regular_light(time))

In [ ]:
def simulate_chang14(idx, time, light, parameters, partial_save_path):
    print(f"Simulating idx: {idx}")
    start_time = lights_on_start_times[idx]
    equilibration_light = regular_light_options[idx]
    # equilibrate
    equilibration_model = Skeldon23(params=parameters)
    initial_condition = equilibration_model.equilibrate(time, equilibration_light, num_loops=10)
    # simulate
    model = Skeldon23(params=parameters)
    trajectory = model.integrate(time, initial_condition=initial_condition,
                                 input=light)
    np.savez(f"{partial_save_path}_lights_on_{start_time}.npz",
             parameters=parameters,
             trajectory_states=trajectory.states,
             sleep=model.sleep_state)

In [ ]:
for ebook_order in ebook_order_options:
    for baseline_lux in baseline_lux_options:
        light = np.load(f"{light_data_location}/chang14_light_indoor_lux_{baseline_lux}_{ebook_order}_light.npy",
                        allow_pickle=True)
        partial_save_path = f"{initial_conditions_save_path}/baseline_lux_{baseline_lux}_{ebook_order}_simulation"

        pool_idx_values = range(len(lights_on_start_times))

        print(f"Simulating pool for {baseline_lux} lux, {ebook_order}")
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(lambda idx: simulate_chang14(
                idx, time, light, parameters, partial_save_path
            ), pool_idx_values))

# Simulate pulses of light at different intervals after wake up

In [ ]:
light_pulse_save_path = '../data/light_pulse_simulations/'

if not os.path.exists(light_pulse_save_path):
    os.makedirs(light_pulse_save_path)

In [ ]:
def alph_forger(light):
    I0 = 9500.0
    p = .6
    a0 = .16
    return a0 * (np.power(light, p) / np.power(I0, p))


def skeldon_model(u, light, current_sleep_state):
    x = u[0]
    xc = u[1]
    n = u[2]
    H = u[3]
    S = current_sleep_state

    effective_light = (1.0 - S) * light
    tx = 24.2
    G = 19.875
    k = .55
    mu = .23
    b = 0.013
    mu_sleep = 17.78
    chi = 45.0

    Bh = G * (1 - n) * alph_forger(effective_light)
    B = Bh * (1 - .4 * x) * (1 - .4 * xc)

    dydt = [0, 0, 0, 0]

    dydt[0] = np.pi / 12.0 * (xc + B)
    dydt[1] = np.pi / 12.0 * (
        mu * (xc - 4.0 * np.power(xc, 3.0) / 3.0) - x * (np.power((24.0 / (.99669 * tx)), 2.0) + k * B))
    dydt[2] = 60.0 * (alph_forger(effective_light) * (1.0 - n) - b * n)
    dydt[3] = (1.0 / chi) * (- H + (1.0 - S) * mu_sleep)

    return np.array(dydt)
    

def skeldon_rk4_step(ustart: np.ndarray, light_val: float, current_sleep: int, dt: float):
    k1 = skeldon_model(ustart, light_val, current_sleep)
    k2 = skeldon_model(ustart + dt / 2 * k1, light_val,
                       current_sleep)
    k3 = skeldon_model(ustart + dt / 2 * k2, light_val,
                       current_sleep)
    k4 = skeldon_model(ustart + dt * k3, light_val,
                       current_sleep)
    return ustart + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)


def circadian_modulation_of_sleep(x, xc):
    c20 = 0.7896
    alpha21 = -0.3912
    alpha22 = 0.7583
    beta21 = -0.4442
    beta22 = 0.0250
    beta23 = -0.9647

    linear_term = c20 + alpha21 * xc + alpha22 * x
    quadratic_term = beta21 * xc * xc + beta22 * xc * x + beta23 * x * x
    C = linear_term + quadratic_term
    return C


def H_thresholds(x, xc):
    H0 = 13.0
    Delta = 1.0
    ca = 1.72
    C = circadian_modulation_of_sleep(x, xc)
    H_plus = H0 + 0.5 * Delta + ca * C
    H_minus = H0 - 0.5 * Delta + ca * C
    return H_plus, H_minus


def run_model(waking_light_level=3000, pulse_light_level=5000, pulse_at=12, pulse_duration=1, 
              initial_condition=np.array([-0.504, -0.8816, 0.001779, 13.28]),
              initial_sleep_state=0.0, simulation_days=20, hours_per_day=24):
    time_spent_awake = 0

    def wake_dependent_light_intensity(time_spent_awake, pulse_at=pulse_at, pulse_duration=pulse_duration):
        if time_spent_awake < pulse_at:
            return 0
        elif time_spent_awake < pulse_at + pulse_duration:
            return pulse_light_level
        else:
            return 0

    def light_intensity(t):
        t = np.mod(t, 24)
        if 6 <= t < 8:
            return waking_light_level * (t - 6) / 2
        elif 8 <= t < 18:
            return waking_light_level
        elif 18 <= t < 20:
            return waking_light_level * (20 - t) / 2
        else:
            return 0

    dt = 0.001

    count = 0
    num_days = simulation_days
    simulation_duration = hours_per_day * num_days

    u0 = initial_condition
    sol = np.zeros((4, int(simulation_duration / dt)))
    sol[:, 0] = u0
    current_sleep_state = initial_sleep_state

    light_history = []
    sleep_history = [current_sleep_state]

    time_history = [-dt]
    for k in range(int(simulation_duration / dt) - 1):

        light_value = light_intensity(k * dt)
        pulse = wake_dependent_light_intensity(
            time_spent_awake=time_spent_awake)
        light_value = light_value + pulse

        u0 = skeldon_rk4_step(
            u0, light_value, current_sleep=current_sleep_state, dt=dt)

        sol[:, k + 1] = u0
        S = current_sleep_state
        new_sleep_state = current_sleep_state
        H = sol[3, k+1]
        H_plus, H_minus = H_thresholds(sol[0, k+1], sol[1, k+1])

        if S == 0:
            if H >= H_plus:
                new_sleep_state = 1.0
                # print(f"Day {count}: Fell asleep at {np.mod(k*dt, 24)}")
                count = count + 1

        else:
            if H <= H_minus:
                new_sleep_state = 0.0
                # print(f"Day {count}: Woke up at {np.mod(k*dt, 24)}")

        forced_wakeup_by_light = False
        if forced_wakeup_by_light:
            new_sleep_state = 0.0

        if current_sleep_state == 0:
            time_spent_awake = time_spent_awake + dt
        else:
            time_spent_awake = 0
        current_sleep_state = new_sleep_state
        light_history.append((1 - current_sleep_state)
                             * light_value)
        sleep_history.append(current_sleep_state)
        time_history.append(k * dt)

    return sol, sleep_history, light_history, dt, num_days

In [ ]:
def simulate_light_pulse(lux_pulse_time_tuple):
    print(f"Simulating baseline lux: {baseline_lux}, pulse at: {pulse_at}")
    baseline_lux = lux_pulse_time_tuple[0]
    pulse_at = lux_pulse_time_tuple[1]
    # Equilibration
    eq_sol, eq_sleep_history, _, dt, num_days = run_model(waking_light_level=baseline_lux, pulse_at=pulse_at,
                                                                            simulation_days=10)
    initial_condition=eq_sol[:, -1]
    initial_sleep_state = int(eq_sleep_history[-1])
    # Simulation
    sol, sleep_history, light_history, dt, num_days = run_model(waking_light_level=baseline_lux, pulse_at=pulse_at, 
                                                                initial_condition=initial_condition,
                                                                initial_sleep_state=initial_sleep_state)
    np.savez(f'{light_pulse_save_path}/baseline_{baseline_lux}_pulse_at_{pulse_at}.npz', 
                sol=sol, sleep_history=sleep_history, light_history=light_history, dt=dt, num_days=num_days)

In [ ]:
baseline_lux_values = np.array([0, 25, 50, 75, 100, 250, 500, 750, 1000], dtype=float)
pulse_at_values = np.linspace(0, 14, 17)

tuple_values = list(product(baseline_lux_values, pulse_at_values))

with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda idx: simulate_light_pulse(
        tuple_values[idx]
    ), range(len(tuple_values))))